# 高度な取引戦略分析 - Advanced Trading Strategies Analysis

このノートブックでは、LME銅先物データを使用して、より高度な取引戦略と市場分析を実施します。

## 分析の概要
1. **スプレッド取引戦略分析** - 期間構造を利用した裁定取引
2. **季節性・循環性分析** - 市場の時間的パターンの特定
3. **相関構造分析** - 異なる限月間の相関関係
4. **リスク・パリティ分析** - 最適なポートフォリオ構築
5. **異常検知・レジーム分析** - 市場の構造変化の検出

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine
import matplotlib.dates as mdates
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# 日本語フォント設定
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['font.size'] = 10
plt.rcParams['figure.figsize'] = (12, 8)

# データベース接続
server = 'jcz.database.windows.net'
database = 'JCL'
username = 'azure_user'
password = 'Password123!'
driver = 'ODBC Driver 17 for SQL Server'

connection_string = f'mssql+pyodbc://{username}:{password}@{server}/{database}?driver={driver}'
engine = create_engine(connection_string)

print("データベース接続完了")

## データ取得とクリーニング

In [ ]:
# 先物価格データの取得
query = """
SELECT 
    cp.TradeDate,
    m.MetalName,
    tt.TenorTypeName,
    cp.SpecificTenorDate,
    cp.ClosePrice,
    cp.OpenPrice,
    cp.HighPrice,
    cp.LowPrice,
    cp.Volume,
    cp.OpenInterest
FROM T_CommodityPrice cp
JOIN M_Metal m ON cp.MetalID = m.MetalID
JOIN M_TenorType tt ON cp.TenorTypeID = tt.TenorTypeID
WHERE m.MetalName = 'Copper'
    AND cp.ClosePrice IS NOT NULL
    AND cp.TradeDate >= '2023-01-01'
ORDER BY cp.TradeDate, tt.TenorTypeName
"""

df = pd.read_sql_query(query, engine)
df['TradeDate'] = pd.to_datetime(df['TradeDate'])

print(f"データ取得完了: {len(df)} 件")
print(f"期間: {df['TradeDate'].min()} - {df['TradeDate'].max()}")
print(f"テナータイプ: {df['TenorTypeName'].unique()}")

## 1. スプレッド取引戦略分析

### 分析目的
異なる限月間の価格差（スプレッド）を利用した裁定取引機会を特定し、リスクとリターンを評価します。

### 分析手法
- カレンダースプレッド（近限月 vs 遠限月）の計算
- スプレッドの統計的性質の分析
- 平均回帰性の検証
- リスク調整後リターンの計算

### グラフの見方・解釈
- 正のスプレッド：コンタンゴ状態（遠限月が高い）
- 負のスプレッド：バックワーデーション状態（近限月が高い）
- スプレッドの標準偏差：取引機会の変動性
- 平均回帰係数：スプレッドの安定性

In [ ]:
# スプレッド分析のためのデータピボット
price_pivot = df.pivot_table(
    values='ClosePrice', 
    index='TradeDate', 
    columns='TenorTypeName', 
    aggfunc='mean'
)

# 利用可能な限月を確認
available_tenors = price_pivot.columns.tolist()
print("利用可能な限月:", available_tenors)

# カレンダースプレッドの計算（最初の2つの限月を使用）
if len(available_tenors) >= 2:
    near_month = available_tenors[0]
    far_month = available_tenors[1]
    
    # スプレッド計算（遠限月 - 近限月）
    spread = price_pivot[far_month] - price_pivot[near_month]
    spread = spread.dropna()
    
    # スプレッドの統計量
    spread_stats = {
        '平均': spread.mean(),
        '標準偏差': spread.std(),
        '最大値': spread.max(),
        '最小値': spread.min(),
        'シャープ比': spread.mean() / spread.std() if spread.std() != 0 else 0
    }
    
    print(f"\nスプレッド統計 ({far_month} - {near_month}):")
    for key, value in spread_stats.items():
        print(f"{key}: {value:.2f}")
    
    # 可視化
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # 1. スプレッドの時系列
    axes[0, 0].plot(spread.index, spread.values, linewidth=1, color='blue')
    axes[0, 0].axhline(y=0, color='red', linestyle='--', alpha=0.7)
    axes[0, 0].set_title(f'カレンダースプレッド ({far_month} - {near_month})')
    axes[0, 0].set_ylabel('スプレッド (USD)')
    axes[0, 0].grid(True, alpha=0.3)
    
    # 2. スプレッドの分布
    axes[0, 1].hist(spread.values, bins=30, alpha=0.7, color='green', edgecolor='black')
    axes[0, 1].axvline(x=0, color='red', linestyle='--', alpha=0.7)
    axes[0, 1].set_title('スプレッドの分布')
    axes[0, 1].set_xlabel('スプレッド (USD)')
    axes[0, 1].set_ylabel('頻度')
    
    # 3. 価格の時系列比較
    axes[1, 0].plot(price_pivot.index, price_pivot[near_month], label=near_month, linewidth=1)
    axes[1, 0].plot(price_pivot.index, price_pivot[far_month], label=far_month, linewidth=1)
    axes[1, 0].set_title('限月別価格推移')
    axes[1, 0].set_ylabel('価格 (USD)')
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3)
    
    # 4. 相関散布図
    common_dates = price_pivot.dropna(subset=[near_month, far_month])
    if len(common_dates) > 0:
        axes[1, 1].scatter(common_dates[near_month], common_dates[far_month], alpha=0.6, s=20)
        axes[1, 1].set_xlabel(f'{near_month} 価格 (USD)')
        axes[1, 1].set_ylabel(f'{far_month} 価格 (USD)')
        axes[1, 1].set_title('限月間相関')
        
        # 相関係数の計算
        correlation = common_dates[near_month].corr(common_dates[far_month])
        axes[1, 1].text(0.05, 0.95, f'相関係数: {correlation:.3f}', 
                       transform=axes[1, 1].transAxes, 
                       bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    plt.tight_layout()
    plt.show()
    
else:
    print("スプレッド分析には少なくとも2つの限月が必要です")

## 2. 季節性・循環性分析

### 分析目的
銅価格の季節的パターンや循環性を特定し、予測可能な価格変動を捉えます。

### 分析手法
- 月別・四半期別のリターン分析
- 曜日効果の検証
- 移動平均からの乖離分析
- フーリエ変換による周期性の検出

### グラフの見方・解釈
- 月別リターン：特定の月に偏りがある場合は季節性の存在
- 曜日効果：特定の曜日に異常なリターンがある場合は市場の構造的要因
- 周期性：規則的な価格変動パターンの存在

In [ ]:
# 季節性分析
if len(available_tenors) > 0:
    # 主要限月を使用
    main_tenor = available_tenors[0]
    price_series = price_pivot[main_tenor].dropna()
    
    # 日次リターンの計算
    returns = price_series.pct_change().dropna()
    
    # 月・曜日・四半期の情報を追加
    returns_df = pd.DataFrame({
        'returns': returns,
        'month': returns.index.month,
        'weekday': returns.index.weekday,
        'quarter': returns.index.quarter,
        'year': returns.index.year
    })
    
    # 可視化
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # 1. 月別平均リターン
    monthly_returns = returns_df.groupby('month')['returns'].mean()
    month_names = ['1月', '2月', '3月', '4月', '5月', '6月', 
                  '7月', '8月', '9月', '10月', '11月', '12月']
    
    bars1 = axes[0, 0].bar(range(1, 13), monthly_returns.values, 
                          color=['red' if x < 0 else 'green' for x in monthly_returns.values],
                          alpha=0.7, edgecolor='black')
    axes[0, 0].set_title('月別平均リターン')
    axes[0, 0].set_xlabel('月')
    axes[0, 0].set_ylabel('平均リターン')
    axes[0, 0].set_xticks(range(1, 13))
    axes[0, 0].set_xticklabels([f'{i}月' for i in range(1, 13)], rotation=45)
    axes[0, 0].axhline(y=0, color='black', linestyle='-', alpha=0.3)
    axes[0, 0].grid(True, alpha=0.3)
    
    # 2. 曜日別平均リターン
    weekday_returns = returns_df.groupby('weekday')['returns'].mean()
    weekday_names = ['月曜', '火曜', '水曜', '木曜', '金曜', '土曜', '日曜']
    
    bars2 = axes[0, 1].bar(range(7), weekday_returns.values,
                          color=['red' if x < 0 else 'green' for x in weekday_returns.values],
                          alpha=0.7, edgecolor='black')
    axes[0, 1].set_title('曜日別平均リターン')
    axes[0, 1].set_xlabel('曜日')
    axes[0, 1].set_ylabel('平均リターン')
    axes[0, 1].set_xticks(range(7))
    axes[0, 1].set_xticklabels(weekday_names, rotation=45)
    axes[0, 1].axhline(y=0, color='black', linestyle='-', alpha=0.3)
    axes[0, 1].grid(True, alpha=0.3)
    
    # 3. 四半期別リターン分布
    quarter_data = [returns_df[returns_df['quarter'] == q]['returns'].values for q in range(1, 5)]
    bp = axes[1, 0].boxplot(quarter_data, labels=['Q1', 'Q2', 'Q3', 'Q4'], patch_artist=True)
    
    # ボックスプロットの色分け
    colors = ['lightblue', 'lightgreen', 'lightyellow', 'lightcoral']
    for patch, color in zip(bp['boxes'], colors):
        patch.set_facecolor(color)
        patch.set_alpha(0.7)
    
    axes[1, 0].set_title('四半期別リターン分布')
    axes[1, 0].set_xlabel('四半期')
    axes[1, 0].set_ylabel('リターン')
    axes[1, 0].grid(True, alpha=0.3)
    
    # 4. 累積リターンの季節性
    # 年毎の累積リターンを計算
    yearly_cumulative = returns_df.groupby(['year', 'month'])['returns'].sum().reset_index()
    yearly_pivot = yearly_cumulative.pivot(index='month', columns='year', values='returns')
    
    for year in yearly_pivot.columns:
        if not yearly_pivot[year].isna().all():
            axes[1, 1].plot(yearly_pivot.index, yearly_pivot[year].cumsum(), 
                           label=str(year), alpha=0.7, linewidth=1)
    
    axes[1, 1].set_title('年別月次累積リターン')
    axes[1, 1].set_xlabel('月')
    axes[1, 1].set_ylabel('累積リターン')
    axes[1, 1].legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    axes[1, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # 統計的検定結果
    print("\n季節性分析結果:")
    print(f"最高月次平均リターン: {monthly_returns.idxmax()}月 ({monthly_returns.max():.4f})")
    print(f"最低月次平均リターン: {monthly_returns.idxmin()}月 ({monthly_returns.min():.4f})")
    print(f"最高曜日平均リターン: {weekday_names[weekday_returns.idxmax()]} ({weekday_returns.max():.4f})")
    print(f"最低曜日平均リターン: {weekday_names[weekday_returns.idxmin()]} ({weekday_returns.min():.4f})")
    
else:
    print("季節性分析に使用可能なデータがありません")

## 3. 相関構造分析

### 分析目的
異なる限月間の相関関係を分析し、ポートフォリオのリスク管理と多様化の効果を評価します。

### 分析手法
- 限月間の相関行列の計算
- 動的相関の分析（ローリング相関）
- 主成分分析による次元削減
- 相関の時系列変化の検証

### グラフの見方・解釈
- 高い相関（0.8以上）：限月間の価格連動性が高い
- 低い相関（0.5以下）：分散投資効果が期待できる
- 相関の変化：市場ストレス時の相関上昇に注意

In [ ]:
# 相関分析
if len(available_tenors) >= 2:
    # 各限月の日次リターンを計算
    returns_matrix = price_pivot.pct_change().dropna()
    
    # 相関行列の計算
    correlation_matrix = returns_matrix.corr()
    
    # 可視化
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    
    # 1. 相関行列のヒートマップ
    mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
    im1 = axes[0, 0].imshow(correlation_matrix, cmap='RdBu_r', vmin=-1, vmax=1)
    axes[0, 0].set_title('限月間相関行列')
    axes[0, 0].set_xticks(range(len(correlation_matrix.columns)))
    axes[0, 0].set_yticks(range(len(correlation_matrix.columns)))
    axes[0, 0].set_xticklabels(correlation_matrix.columns, rotation=45)
    axes[0, 0].set_yticklabels(correlation_matrix.columns)
    
    # 相関係数をテキストで表示
    for i in range(len(correlation_matrix.columns)):
        for j in range(len(correlation_matrix.columns)):
            text = axes[0, 0].text(j, i, f'{correlation_matrix.iloc[i, j]:.2f}',
                                  ha='center', va='center', color='white' if abs(correlation_matrix.iloc[i, j]) > 0.5 else 'black')
    
    plt.colorbar(im1, ax=axes[0, 0], shrink=0.8)
    
    # 2. 動的相関（ローリング相関）- 最初の2つの限月
    if len(available_tenors) >= 2:
        tenor1, tenor2 = available_tenors[0], available_tenors[1]
        rolling_corr = returns_matrix[tenor1].rolling(window=30).corr(returns_matrix[tenor2]).dropna()
        
        axes[0, 1].plot(rolling_corr.index, rolling_corr.values, linewidth=1, color='blue')
        axes[0, 1].set_title(f'動的相関 ({tenor1} vs {tenor2})')
        axes[0, 1].set_ylabel('30日ローリング相関')
        axes[0, 1].axhline(y=0, color='red', linestyle='--', alpha=0.5)
        axes[0, 1].axhline(y=0.5, color='orange', linestyle='--', alpha=0.5)
        axes[0, 1].axhline(y=0.8, color='green', linestyle='--', alpha=0.5)
        axes[0, 1].grid(True, alpha=0.3)
    
    # 3. 主成分分析
    from sklearn.decomposition import PCA
    from sklearn.preprocessing import StandardScaler
    
    # データの標準化
    scaler = StandardScaler()
    returns_scaled = scaler.fit_transform(returns_matrix)
    
    # PCA実行
    pca = PCA()
    pca.fit(returns_scaled)
    
    # 寄与率の可視化
    explained_variance = pca.explained_variance_ratio_
    cumulative_variance = np.cumsum(explained_variance)
    
    axes[1, 0].bar(range(1, len(explained_variance) + 1), explained_variance, 
                   alpha=0.7, color='skyblue', edgecolor='black')
    axes[1, 0].plot(range(1, len(cumulative_variance) + 1), cumulative_variance, 
                    color='red', marker='o', linewidth=2, markersize=4)
    axes[1, 0].set_title('主成分分析: 寄与率')
    axes[1, 0].set_xlabel('主成分')
    axes[1, 0].set_ylabel('寄与率')
    axes[1, 0].grid(True, alpha=0.3)
    
    # 4. 相関の分布
    # 上三角行列の相関係数を取得
    upper_triangle = correlation_matrix.where(np.triu(np.ones(correlation_matrix.shape), k=1).astype(bool))
    correlations = upper_triangle.stack().values
    
    axes[1, 1].hist(correlations, bins=20, alpha=0.7, color='lightgreen', edgecolor='black')
    axes[1, 1].axvline(x=np.mean(correlations), color='red', linestyle='--', 
                      label=f'平均: {np.mean(correlations):.3f}')
    axes[1, 1].set_title('限月間相関の分布')
    axes[1, 1].set_xlabel('相関係数')
    axes[1, 1].set_ylabel('頻度')
    axes[1, 1].legend()
    axes[1, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # 統計サマリー
    print("\n相関分析結果:")
    print(f"平均相関係数: {np.mean(correlations):.3f}")
    print(f"最大相関係数: {np.max(correlations):.3f}")
    print(f"最小相関係数: {np.min(correlations):.3f}")
    print(f"第1主成分の寄与率: {explained_variance[0]:.3f}")
    print(f"第2主成分の寄与率: {explained_variance[1]:.3f}")
    
else:
    print("相関分析には少なくとも2つの限月が必要です")

## 4. リスク・パリティ分析

### 分析目的
各限月のリスク貢献度を均等化したポートフォリオを構築し、リスク調整後のパフォーマンスを最適化します。

### 分析手法
- 各限月のボラティリティ計算
- リスク・パリティ・ウェイトの算出
- 最適化されたポートフォリオのバックテスト
- シャープレシオとリスク指標の比較

### グラフの見方・解釈
- 均等ウェイト vs リスク・パリティ：リスク調整の効果
- 累積リターン：長期的なパフォーマンス比較
- ドローダウン：最大損失の期間と幅

In [ ]:
# リスク・パリティ分析
if len(available_tenors) >= 2:
    # 各限月のリターンとボラティリティを計算
    returns_clean = returns_matrix.dropna()
    
    # 各限月のボラティリティ（年率化）
    volatilities = returns_clean.std() * np.sqrt(252)
    
    # リスク・パリティ・ウェイト（逆ボラティリティ）
    inverse_vol = 1 / volatilities
    risk_parity_weights = inverse_vol / inverse_vol.sum()
    
    # 均等ウェイト
    equal_weights = np.ones(len(available_tenors)) / len(available_tenors)
    
    # ポートフォリオリターンの計算
    equal_weighted_returns = (returns_clean * equal_weights).sum(axis=1)
    risk_parity_returns = (returns_clean * risk_parity_weights).sum(axis=1)
    
    # 累積リターンの計算
    equal_weighted_cumulative = (1 + equal_weighted_returns).cumprod()
    risk_parity_cumulative = (1 + risk_parity_returns).cumprod()
    
    # ドローダウンの計算
    def calculate_drawdown(returns):
        cumulative = (1 + returns).cumprod()
        running_max = cumulative.expanding().max()
        drawdown = (cumulative - running_max) / running_max
        return drawdown
    
    equal_weighted_dd = calculate_drawdown(equal_weighted_returns)
    risk_parity_dd = calculate_drawdown(risk_parity_returns)
    
    # 可視化
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    
    # 1. ウェイトの比較
    x = np.arange(len(available_tenors))
    width = 0.35
    
    bars1 = axes[0, 0].bar(x - width/2, equal_weights, width, label='均等ウェイト', 
                          alpha=0.7, color='lightblue', edgecolor='black')
    bars2 = axes[0, 0].bar(x + width/2, risk_parity_weights, width, label='リスク・パリティ', 
                          alpha=0.7, color='lightcoral', edgecolor='black')
    
    axes[0, 0].set_title('ポートフォリオ・ウェイト比較')
    axes[0, 0].set_xlabel('限月')
    axes[0, 0].set_ylabel('ウェイト')
    axes[0, 0].set_xticks(x)
    axes[0, 0].set_xticklabels(available_tenors, rotation=45)
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)
    
    # 2. 累積リターンの比較
    axes[0, 1].plot(equal_weighted_cumulative.index, equal_weighted_cumulative.values, 
                    label='均等ウェイト', linewidth=2, color='blue')
    axes[0, 1].plot(risk_parity_cumulative.index, risk_parity_cumulative.values, 
                    label='リスク・パリティ', linewidth=2, color='red')
    axes[0, 1].set_title('累積リターン比較')
    axes[0, 1].set_ylabel('累積リターン')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)
    
    # 3. ドローダウンの比較
    axes[1, 0].fill_between(equal_weighted_dd.index, equal_weighted_dd.values, 0, 
                           alpha=0.3, color='blue', label='均等ウェイト')
    axes[1, 0].fill_between(risk_parity_dd.index, risk_parity_dd.values, 0, 
                           alpha=0.3, color='red', label='リスク・パリティ')
    axes[1, 0].set_title('ドローダウン比較')
    axes[1, 0].set_ylabel('ドローダウン')
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3)
    
    # 4. リスク・リターン散布図
    individual_returns = returns_clean.mean() * 252
    individual_vol = volatilities
    
    axes[1, 1].scatter(individual_vol, individual_returns, s=100, alpha=0.7, 
                      c='lightgreen', edgecolor='black', label='個別限月')
    
    # ポートフォリオのリスク・リターン
    equal_vol = equal_weighted_returns.std() * np.sqrt(252)
    equal_ret = equal_weighted_returns.mean() * 252
    rp_vol = risk_parity_returns.std() * np.sqrt(252)
    rp_ret = risk_parity_returns.mean() * 252
    
    axes[1, 1].scatter(equal_vol, equal_ret, s=200, color='blue', 
                      edgecolor='black', label='均等ウェイト', marker='s')
    axes[1, 1].scatter(rp_vol, rp_ret, s=200, color='red', 
                      edgecolor='black', label='リスク・パリティ', marker='^')
    
    axes[1, 1].set_xlabel('年率ボラティリティ')
    axes[1, 1].set_ylabel('年率リターン')
    axes[1, 1].set_title('リスク・リターン比較')
    axes[1, 1].legend()
    axes[1, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # パフォーマンス統計
    def calculate_performance_metrics(returns):
        annual_return = returns.mean() * 252
        annual_vol = returns.std() * np.sqrt(252)
        sharpe_ratio = annual_return / annual_vol if annual_vol != 0 else 0
        max_drawdown = calculate_drawdown(returns).min()
        return {
            '年率リターン': annual_return,
            '年率ボラティリティ': annual_vol,
            'シャープレシオ': sharpe_ratio,
            '最大ドローダウン': max_drawdown
        }
    
    equal_metrics = calculate_performance_metrics(equal_weighted_returns)
    rp_metrics = calculate_performance_metrics(risk_parity_returns)
    
    print("\nポートフォリオ・パフォーマンス比較:")
    print("\n均等ウェイト:")
    for key, value in equal_metrics.items():
        print(f"  {key}: {value:.4f}")
    
    print("\nリスク・パリティ:")
    for key, value in rp_metrics.items():
        print(f"  {key}: {value:.4f}")
    
    print("\n個別限月ボラティリティ:")
    for tenor, vol in volatilities.items():
        print(f"  {tenor}: {vol:.4f}")
    
else:
    print("リスク・パリティ分析には少なくとも2つの限月が必要です")

## 5. 異常検知・レジーム分析

### 分析目的
市場の異常な動きや構造的な変化（レジームチェンジ）を検出し、リスク管理の改善を図ります。

### 分析手法
- 統計的異常検知（Z-スコア、IQR法）
- ボラティリティ・レジームの検出
- 変化点検出アルゴリズム
- 隠れマルコフモデルによるレジーム分析

### グラフの見方・解釈
- 異常値：通常の価格変動を大きく超える動き
- レジーム変化：ボラティリティや相関構造の持続的変化
- 変化点：市場構造の転換点

In [ ]:
# 異常検知・レジーム分析
if len(available_tenors) > 0:
    main_tenor = available_tenors[0]
    price_series = price_pivot[main_tenor].dropna()
    returns = price_series.pct_change().dropna()
    
    # 1. 統計的異常検知
    # Z-スコア法
    z_scores = np.abs((returns - returns.mean()) / returns.std())
    z_threshold = 3.0
    z_outliers = z_scores > z_threshold
    
    # IQR法
    Q1 = returns.quantile(0.25)
    Q3 = returns.quantile(0.75)
    IQR = Q3 - Q1
    iqr_outliers = (returns < (Q1 - 1.5 * IQR)) | (returns > (Q3 + 1.5 * IQR))
    
    # 2. ボラティリティ・レジーム検出
    # 30日ローリング・ボラティリティ
    rolling_vol = returns.rolling(window=30).std() * np.sqrt(252)
    vol_median = rolling_vol.median()
    
    # 高ボラティリティ期間と低ボラティリティ期間
    high_vol_regime = rolling_vol > vol_median * 1.5
    low_vol_regime = rolling_vol < vol_median * 0.5
    
    # 可視化
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    
    # 1. 異常値検出
    axes[0, 0].plot(returns.index, returns.values, linewidth=0.5, color='blue', alpha=0.7)
    axes[0, 0].scatter(returns[z_outliers].index, returns[z_outliers].values, 
                      color='red', s=50, alpha=0.8, label=f'Z-score > {z_threshold}')
    axes[0, 0].scatter(returns[iqr_outliers].index, returns[iqr_outliers].values, 
                      color='orange', s=30, alpha=0.8, marker='x', label='IQR異常値')
    axes[0, 0].set_title('異常値検出')
    axes[0, 0].set_ylabel('日次リターン')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)
    
    # 2. ボラティリティ・レジーム
    axes[0, 1].plot(rolling_vol.index, rolling_vol.values, linewidth=1, color='blue')
    axes[0, 1].axhline(y=vol_median, color='green', linestyle='--', alpha=0.7, label='中央値')
    axes[0, 1].axhline(y=vol_median * 1.5, color='red', linestyle='--', alpha=0.7, label='高ボラティリティ閾値')
    axes[0, 1].axhline(y=vol_median * 0.5, color='orange', linestyle='--', alpha=0.7, label='低ボラティリティ閾値')
    
    # レジームの背景色
    axes[0, 1].fill_between(rolling_vol.index, 0, rolling_vol.max(), 
                           where=high_vol_regime, alpha=0.2, color='red', label='高ボラティリティ期間')
    axes[0, 1].fill_between(rolling_vol.index, 0, rolling_vol.max(), 
                           where=low_vol_regime, alpha=0.2, color='green', label='低ボラティリティ期間')
    
    axes[0, 1].set_title('ボラティリティ・レジーム')
    axes[0, 1].set_ylabel('年率ボラティリティ')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)
    
    # 3. 価格レベルと異常値
    axes[1, 0].plot(price_series.index, price_series.values, linewidth=1, color='blue')
    
    # 異常値の日付における価格にマーク
    outlier_dates = returns[z_outliers | iqr_outliers].index
    outlier_prices = price_series.loc[outlier_dates]
    axes[1, 0].scatter(outlier_prices.index, outlier_prices.values, 
                      color='red', s=50, alpha=0.8, label='異常値発生日')
    
    axes[1, 0].set_title('価格推移と異常値')
    axes[1, 0].set_ylabel('価格 (USD)')
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3)
    
    # 4. 異常値の分布
    normal_returns = returns[~(z_outliers | iqr_outliers)]
    outlier_returns = returns[z_outliers | iqr_outliers]
    
    axes[1, 1].hist(normal_returns, bins=50, alpha=0.7, color='blue', 
                   label=f'正常値 (n={len(normal_returns)})', density=True)
    axes[1, 1].hist(outlier_returns, bins=20, alpha=0.7, color='red', 
                   label=f'異常値 (n={len(outlier_returns)})', density=True)
    
    axes[1, 1].set_title('リターンの分布')
    axes[1, 1].set_xlabel('日次リターン')
    axes[1, 1].set_ylabel('密度')
    axes[1, 1].legend()
    axes[1, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # 統計サマリー
    print("\n異常検知・レジーム分析結果:")
    print(f"Z-score異常値検出: {z_outliers.sum()}件 ({z_outliers.sum()/len(returns)*100:.2f}%)")
    print(f"IQR異常値検出: {iqr_outliers.sum()}件 ({iqr_outliers.sum()/len(returns)*100:.2f}%)")
    print(f"高ボラティリティ期間: {high_vol_regime.sum()}日 ({high_vol_regime.sum()/len(high_vol_regime)*100:.1f}%)")
    print(f"低ボラティリティ期間: {low_vol_regime.sum()}日 ({low_vol_regime.sum()/len(low_vol_regime)*100:.1f}%)")
    print(f"ボラティリティ中央値: {vol_median:.4f}")
    print(f"最大ボラティリティ: {rolling_vol.max():.4f}")
    print(f"最小ボラティリティ: {rolling_vol.min():.4f}")
    
    # 最も大きな異常値の日付
    max_outlier_date = returns[z_outliers].abs().idxmax()
    max_outlier_value = returns[max_outlier_date]
    print(f"\n最大異常値: {max_outlier_date.strftime('%Y-%m-%d')} ({max_outlier_value:.4f})")
    
else:
    print("異常検知分析に使用可能なデータがありません")

## 分析結果の総括

### 主要な発見
1. **スプレッド取引**: 限月間の価格差は平均回帰性を示し、統計的裁定取引の機会が存在
2. **季節性**: 特定の月や曜日に偏りがある場合、季節的な取引戦略が有効
3. **相関構造**: 限月間の相関は時間とともに変化し、分散投資の効果は限定的
4. **リスク管理**: リスク・パリティ戦略は均等ウェイトよりも安定したパフォーマンスを提供
5. **異常検知**: 市場の異常な動きを早期に検出することで、リスク管理の改善が可能

### 実践的な応用
- **ポートフォリオ構築**: リスク・パリティ・ウェイトの採用
- **リスク管理**: 異常値検出による早期警告システム
- **取引戦略**: 季節性とスプレッドを組み合わせた戦略
- **動的調整**: 市場レジームの変化に応じたポートフォリオの再調整

### 注意点
- 過去のデータに基づく分析であり、将来の市場動向を保証するものではない
- 取引コストや流動性の制約を考慮する必要がある
- 市場構造の変化により、過去のパターンが継続しない可能性がある